# HCR Pre-Processing


## *** Author: Keshava Prasad Gubbi***
## Script for pre-processing HCR Lines.
## Read the filenames and split and determine names of sig1, sig2 and ref channels
## Split the Czi imagefilename into respective channels and save it as nrrd with respective names as per channels.

In [3]:
import os
import numpy as np
from aicspylibczi import CziFile
import re
import nrrd

### Filetype A (with ch2 as reference Channel) - For the old style of naming images!

### Please enter the path (full folder path into the file_path variable) for the line to be pre-processed!

In [3]:
file_path = r'C:\Users\keshavgubbi\Desktop\HCR\raw_data\20201204_pmch_pmch1'
ref_channel_path = file_path + '/reference/'
sig_channel_path = file_path + '/signal/'

In [6]:
def create_channel_folder(path):
    if not os.path.exists(path):
        print(f'Creating {path}')
        os.makedirs(path, exist_ok=True)
        return path

### ########Splitting Channels####################

In [5]:

ref_num = int(input('Enter Reference Channel Number: (please enter 1 if refrence channel in image in channel2!)'))
for file in os.listdir(file_path):
    if file.endswith('.czi'):
        print('Image name: ',file)
#         ref_name, exp_name, sig2_name, sig1_name, f_num = get_channel_names(file)
        create_channel_folder(ref_channel_path)
        create_channel_folder(sig_channel_path)
        
        name, ext = file.split('.',1)
#         print(name)
        e_name,  ref_ch_name, sig_ch1_name, fish_num = re.split(r'_ch\d_', name)
        embryo, sig_ch2_name = e_name.rsplit('_',1)
        
        print('fish_num:', fish_num)
        print('ref_ch_name:', ref_ch_name)
        print('sig_ch1_name:', sig_ch1_name)
        print('sig_ch2_name:', sig_ch2_name)
#         print('embryo_name:', embryo_name)
        print('embryo:', embryo)

        c = CziFile(os.path.join(file_path, file))
        max_channels = range(*c.dims_shape()[0]['C'])
        max_slices = range(*c.dims_shape()[0]['Z'])

        # for ch_num in max_channels:
        for ch_num in range(len(max_channels)):
            print('Processing ch_num:', ch_num, '...........')
            ref_image_list = sig1_image_list = sig2_image_list = []
            for z_plane in max_slices:
                imgarray, shp = c.read_image(B=0, S=0, C=ch_num, T=0, Z=z_plane)
                if ch_num == 0:
                    ref_image_list.append(np.squeeze(imgarray))
                elif ch_num == 1:
                    sig1_image_list.append(np.squeeze(imgarray))
                elif ch_num == 2:
                    sig2_image_list.append(np.squeeze(imgarray))   
                else:
                    print('Exceeded Maximum number of channels')
        RImage = np.stack(ref_image_list).astype('uint8')

        S1Image = np.stack(sig1_image_list).astype('uint8')

        S2Image = np.stack(sig1_image_list).astype('uint8')
        
        ### Writing the individual Channels into nrrd format
        print('Writing the individual Channels into nrrd format!')
        
        print(f'Creating Reference Channel nrrd file with name : {embryo}_{fish_num}_{ref_ch_name}.nrrd ')
        nrrd.write(os.path.join(ref_channel_path, f"{embryo}_{fish_num}_{ref_ch_name}.nrrd"), RImage, index_order='C')
        print(f'Creating Signal1 Channel nrrd file with name : {embryo}_{fish_num}_{sig_ch1_name}.nrrd')
        nrrd.write(os.path.join(sig_channel_path, f"{embryo}_{fish_num}_{sig_ch1_name}.nrrd"),S1Image, index_order='C')
        print(f'Creating Signal2 Channel nrrd file with name : {embryo}_{fish_num}_{sig_ch2_name}.nrrd')
        nrrd.write(os.path.join(sig_channel_path, f"{embryo}_{fish_num}_{sig_ch2_name}.nrrd"),S2Image, index_order='C')
        
        
        print(f'###################### Completed processing {file} ###################### ')

Enter Reference Channel Number: (please enter 1 if refrence channel in image in channel2!)1
Image name:  6dpf_huc_h2b_gcamp6s_pmch1_ch3_GC6s_ch2_pmch2_ch1_1.czi
fish_num: 1
ref_ch_name: GC6s
sig_ch1_name: pmch2
sig_ch2_name: pmch1
embryo: 6dpf_huc_h2b_gcamp6s
Processing ch_num: 0 ...........
Processing ch_num: 1 ...........
Processing ch_num: 2 ...........
Writing the individual Channels into nrrd format!
Creating Reference Channel nrrd file with name : 6dpf_huc_h2b_gcamp6s_1_GC6s.nrrd 
Creating Signal1 Channel nrrd file with name : 6dpf_huc_h2b_gcamp6s_1_pmch2.nrrd
Creating Signal2 Channel nrrd file with name : 6dpf_huc_h2b_gcamp6s_1_pmch1.nrrd
###################### Completed processing 6dpf_huc_h2b_gcamp6s_pmch1_ch3_GC6s_ch2_pmch2_ch1_1.czi ###################### 
Image name:  6dpf_huc_h2b_gcamp6s_pmch1_ch3_GC6s_ch2_pmch2_ch1_2.czi
fish_num: 2
ref_ch_name: GC6s
sig_ch1_name: pmch2
sig_ch2_name: pmch1
embryo: 6dpf_huc_h2b_gcamp6s
Processing ch_num: 0 ...........
Processing ch_num: 1 

### Stacking the indvidual slices into respective images based on channel number!

### Filetype B (with ch1 as reference Channel) - For the new style of naming images!

### Please enter the path (full folder path into the file_path variable) for the line to be pre-processed!

In [4]:
file_path = r'C:\Users\keshavgubbi\Desktop\HCR\raw_data\20210302_rspo1_cckb'
ref_channel_path = file_path + '/reference/'
sig_channel_path = file_path + '/signal/'

In [10]:
########Splitting Channels####################
ref_num = int(input('Enter Reference Channel Number: (please enter 0 if refrence channel in image is channel1!)'))

for file in os.listdir(file_path):
    if file.endswith('.czi'):
        print('Image name: ',file)
        create_channel_folder(ref_channel_path)
        create_channel_folder(sig_channel_path)
        e_name, sig_ch1_name, sig_ch2_name, ref_ch_name = re.split(r'_ch\d_', file)
        c, ext = ref_ch_name.split('.',1)
        ref_ch_name, fish_num = c.split('_',1)
        e1_name, f = e_name.rsplit('_',1)
        days, embryo_name = e1_name.split('_',1)
        
        print('fish_num:', fish_num)
        print('ref_ch_name:', ref_ch_name)
        print('sig_ch1_name:', sig_ch1_name)
        print('sig_ch2_name:', sig_ch2_name)
        print('embryo_name:', embryo_name)
        
#         print(e_name)
#         print(e1_name)
#         print(f, days)
        
        c = CziFile(os.path.join(file_path, file))
        max_channels = range(*c.dims_shape()[0]['C'])
        max_slices = range(*c.dims_shape()[0]['Z'])

        # for ch_num in max_channels:
        for ch_num in range(len(max_channels)):
            print('Processing ch_num:', ch_num, '...........')
            ref_image_list = sig1_image_list = sig2_image_list = []
            for z_plane in max_slices:
                imgarray, shp = c.read_image(B=0, S=0, C=ch_num, T=0, Z=z_plane)
                if ch_num == 0 and ch_num == ref_num:
                    ref_image_list.append(np.squeeze(imgarray))
                elif ch_num == 1:
                    sig1_image_list.append(np.squeeze(imgarray))
                elif ch_num == 2:
                    sig2_image_list.append(np.squeeze(imgarray))   
                else:
                    print('Exceeded Maximum number of channels')
                    
        RImage = np.stack(ref_image_list).astype('uint8')

        S1Image = np.stack(sig1_image_list).astype('uint8')

        S2Image = np.stack(sig1_image_list).astype('uint8')

        ### Writing the individual Channels into nrrd format
        print('Writing the individual Channels into nrrd format!')

        print(f'Creating Reference Channel nrrd file with name : {embryo_name}_{fish_num}_ch0_{ref_ch_name}.nrrd ')
        nrrd.write(os.path.join(ref_channel_path, f"{embryo_name}_{fish_num}_ch0_{ref_ch_name}.nrrd"), RImage, index_order='C')
        print(f'Creating nrrd file for Signal1 Channel with name : {embryo_name}_{fish_num}_ch1_{sig_ch1_name}.nrrd')
        nrrd.write(os.path.join(sig_channel_path, f"{embryo_name}_{fish_num}_ch1_{sig_ch1_name}.nrrd"),S1Image, index_order='C')
        print(f'Creating nrrd file Signal2 Channel with name : {embryo_name}_{fish_num}_ch2_{sig_ch1_name}.nrrd')
        nrrd.write(os.path.join(sig_channel_path, f"{embryo_name}_{fish_num}_ch2_{sig_ch2_name}.nrrd"),S2Image, index_order='C')
        
        print(f'###################### Completed processing {file} ###################### ')

Enter Reference Channel Number: (please enter 0 if refrence channel in image is channel1!) 0


Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_1.czi
fish_num: 1
ref_ch_name: GC6s
sig_ch1_name: cckb
sig_ch2_name: rspo1
embryo_name: huc_h2b
###################### Completed processing 6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_1.czi ###################### 
Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_2.czi
fish_num: 2
ref_ch_name: GC6s
sig_ch1_name: cckb
sig_ch2_name: rspo1
embryo_name: huc_h2b
###################### Completed processing 6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_2.czi ###################### 
Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_3.czi
fish_num: 3
ref_ch_name: GC6s
sig_ch1_name: cckb
sig_ch2_name: rspo1
embryo_name: huc_h2b
###################### Completed processing 6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_3.czi ###################### 
